Этот ноутбук читает данные из формата CONLL и сохраняет их в JSON (что удобно для последующей работы). 

Тут нарушается принцип DRY (последние две ячейки -- копипаста друг друга с изменением значения одной переменной). Но, думаю, это не страшно, все равно по-хорошему этот код в дальнейшем надо будет вынести в скрипт, там и поправим эту копипасту. 

In [1]:
import os

In [2]:
from pathlib import Path

In [3]:
path_to_data = Path('../data')

In [4]:
langdirs = [
    d for d in os.listdir(path_to_data) 
    if '.' not in d  # exclude files and hidden folders
]

In [5]:
langdirs

['RU-Russian',
 'TR-Turkish',
 'FA-Farsi',
 'ZH-Chinese',
 'ES-Spanish',
 'NL-Dutch',
 'BN-Bangla',
 'KO-Korean',
 'DE-German',
 'EN-English',
 'HI-Hindi']

In [6]:
def make_new_sample():
    return {
        'tokens': [],
        'token_labels': [],
        'lang': langdir,
        'id': None,
        'domain': None,
        'type': None
    }

In [ ]:
import json
import re

ID_DOMAIN = re.compile(r'# id (\S+)\sdomain=(\w+)')
# TOKEN_LABEL = re.compile(r'(\S+) _ _ (\S+)')
CONLL_PATTERN = '_ _'

TYPE = 'train'

wrong_lines = []
dataset = []

for langdir in langdirs:
    fname = [f for f in os.listdir(path_to_data / langdir) if TYPE in f][0]
    with (path_to_data / langdir / fname).open() as f:
        lines = f.readlines()

    new_sample = make_new_sample()
    for line in lines:
        line = line.strip()
        if line:
            match = ID_DOMAIN.match(line)
            if match:
                new_sample['id'] = match.group(1)
                new_sample['domain'] = match.group(2)
                new_sample['type'] = TYPE
            elif CONLL_PATTERN in line:
                splitted = line.split(CONLL_PATTERN)
                new_sample['tokens'].append(splitted[0].strip())
                new_sample['token_labels'].append(splitted[1].strip())        
            else:
                wrong_lines.append(line)
        else:
            dataset.append(new_sample)
            new_sample = make_new_sample()        
            
assert not wrong_lines

with open(f'../data/{TYPE}_dataset.json', 'w') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

Аналогично поступим для dev

In [ ]:
import json
import re

ID_DOMAIN = re.compile(r'# id (\S+)\sdomain=(\w+)')
# TOKEN_LABEL = re.compile(r'(\S+) _ _ (\S+)')
CONLL_PATTERN = '_ _'

TYPE = 'dev'

wrong_lines = []
dataset = []

for langdir in langdirs:
    fname = [f for f in os.listdir(path_to_data / langdir) if TYPE in f][0]
    with (path_to_data / langdir / fname).open() as f:
        lines = f.readlines()

    new_sample = make_new_sample()
    for line in lines:
        line = line.strip()
        if line:
            match = ID_DOMAIN.match(line)
            if match:
                new_sample['id'] = match.group(1)
                new_sample['domain'] = match.group(2)
                new_sample['type'] = TYPE
            elif CONLL_PATTERN in line:
                splitted = line.split(CONLL_PATTERN)
                new_sample['tokens'].append(splitted[0].strip())
                new_sample['token_labels'].append(splitted[1].strip())        
            else:
                wrong_lines.append(line)
        else:
            dataset.append(new_sample)
            new_sample = make_new_sample()        
            
assert not wrong_lines

with open(f'../data/{TYPE}_dataset.json', 'w') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)